In [1]:
import tensorflow as tf
import numpy as np
import dbGeneration as db
import random

# Data Pre-Processing

In [2]:
tensorInputs, raw_labels = db.get_tensorinputs_and_labels()
tensorInputs = np.asarray(tensorInputs)

Returned 15790 data points


### Convert labels to one-hot vector

In [3]:
pre_labels = []
for i in raw_labels:
    if i == -1:
        pre_labels.append([1,0])
    else:
        pre_labels.append([0,1])

pre_labels = np.asarray(pre_labels)

In [4]:
print(pre_labels.shape)
print(pre_labels)

(15790, 2)
[[1 0]
 [1 0]
 [1 0]
 ..., 
 [0 1]
 [0 1]
 [0 1]]


### Define training and testing data

In [5]:
train_gameboards = tensorInputs[0:14000]
test_gameboards = tensorInputs[14000:15790]
train_labels = pre_labels[0:14000]
test_labels = pre_labels[14000:15790]

In [6]:
# Re-order the training data, so the batches are more mixed
data = list(zip(train_gameboards, train_labels))
random.shuffle(data)
train_gameboards = []
train_labels = []
for i in data:
    train_gameboards.append([i[0]][0])
    train_labels.append(i[1])

In [7]:
print(test_gameboards[0], test_labels[0])
print()
print(train_gameboards[0], train_labels[0])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  0.  0.  0.  0. -1.
  1.  0.  0.  0.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [1 0]

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -1.  1.  1.  1.  0.  0.  0.  0. -1. -1. -1.  0.  0.  0.  0.  0.  0.  1.
 -1. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [0 1]


# Model Definition

In [8]:
gameboards = tf.placeholder(tf.float32, (None, 64), name="gameBoards")
gameboards_2d = tf.reshape(gameboards, (-1, 8, 8, 1), name="gameBoards_2D")
gameboards_1d = tf.reshape(gameboards, (-1, 64, 1), name="gameBoards_2D")


In [9]:
w = tf.Variable(tf.truncated_normal([64, 2], mean=1.0, stddev=0.5), name="weights")
b = tf.Variable(tf.ones([1,2]), name="bias")

In [10]:
# Add two convolution layers with max pooling
#conv1 = tf.layers.conv1d(gameboards_1d, 32, 5, padding="same", name="Conv1")

## 2D
'''
conv1 = tf.layers.conv2d(gameboards_2d, 64, 5, padding="same", name="Conv1")
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv2d(pool1, 128, 5, padding="same", name="Conv2")
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
pool2_flat = tf.reshape(pool2, (-1, 8*8*8), name="Pool2_Flat")
'''

## 1D
conv1 = tf.layers.conv1d(gameboards_1d, 32, 5, padding="same", name="Conv1")
pool1 = tf.layers.max_pooling1d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv1d(pool1, 200, 5, padding="same", name="Conv2")
pool2 = tf.layers.max_pooling1d(conv2, 2, 2, name="Pool2")

# 50 is batch size, image is 8*8 (?)
pool2_flat = tf.reshape(pool2, (-1, 8*8*50), name="Pool2_Flat")

# A dense layer with dropout
dense = tf.layers.dense(pool2_flat, 512, activation=tf.nn.relu, name="Dense")
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

# The original dense layer to compute logits that are later used for classification
logits = tf.layers.dense(dropout, 2, activation=None, name="Logits")

In [11]:
# We can take a look at the shapes of some tensors
(pool2.shape, pool2_flat.shape, dropout.shape, logits.shape)

(TensorShape([Dimension(None), Dimension(16), Dimension(200)]),
 TensorShape([Dimension(None), Dimension(3200)]),
 TensorShape([Dimension(None), Dimension(512)]),
 TensorShape([Dimension(None), Dimension(2)]))

In [12]:
#logits = tf.add(tf.matmul(gameboards, w), b)

In [13]:
w.shape, b.shape, logits.shape

(TensorShape([Dimension(64), Dimension(2)]),
 TensorShape([Dimension(1), Dimension(2)]),
 TensorShape([Dimension(None), Dimension(2)]))

In [14]:
# None, as in, this is not yet defined, there could be any number of them input. 
# 2, as in, there are two elements in the one-hot vector

labels = tf.placeholder(tf.int32, [None, 2], name="labels")

In [15]:
# This loss is the elementwise loss
#loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# We want the mean loss
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))

In [16]:
loss.shape

TensorShape([])

In [17]:
train = tf.train.AdamOptimizer().minimize(loss)

## Define an error rate to evaluate model performance


In [18]:
with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(
            tf.not_equal(tf.argmax(logits, axis=1), tf.argmax(labels, axis=1)), tf.float32))

In [19]:
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)

<tf.Tensor 'Error_1:0' shape=() dtype=string>

In [20]:
number_of_training_samples = len(list(train_gameboards))
print(number_of_training_samples)

number_of_samples_per_batch = 32

number_of_batches = int(number_of_training_samples / number_of_samples_per_batch)
print(number_of_batches)

14000
437


In [21]:
sess = tf.Session()

In [22]:
sess.run(tf.global_variables_initializer())

In [23]:
print(number_of_batches)

437


In [24]:
print(number_of_samples_per_batch)

32


In [32]:
for epoch in range(5):
    for i in range(number_of_batches):
        first_index = i*number_of_samples_per_batch
        second_index = (i*number_of_samples_per_batch) + number_of_samples_per_batch
        batch = train_gameboards[first_index:second_index]
        batchLabels = train_labels[first_index:second_index]

        if i % 10 == 0:
            Error, Loss = sess.run([error, loss], feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})
            print("Loss: {}, \tError: {}".format(Loss, Error))
        sess.run(train, feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})


Loss: 0.18114687502384186, 	Error: 0.125
Loss: 0.03276273235678673, 	Error: 0.0
Loss: 0.18298836052417755, 	Error: 0.125
Loss: 0.10911297798156738, 	Error: 0.0625
Loss: 0.13843530416488647, 	Error: 0.09375
Loss: 0.11138470470905304, 	Error: 0.09375
Loss: 0.13289068639278412, 	Error: 0.09375
Loss: 0.27651116251945496, 	Error: 0.1875
Loss: 0.05733189731836319, 	Error: 0.03125
Loss: 0.17461490631103516, 	Error: 0.09375
Loss: 0.08577390015125275, 	Error: 0.03125
Loss: 0.06169934570789337, 	Error: 0.03125
Loss: 0.13560457527637482, 	Error: 0.09375
Loss: 0.08085232973098755, 	Error: 0.03125
Loss: 0.1334208995103836, 	Error: 0.09375
Loss: 0.19343867897987366, 	Error: 0.125
Loss: 0.09272146970033646, 	Error: 0.0625
Loss: 0.1548590511083603, 	Error: 0.15625
Loss: 0.11252652853727341, 	Error: 0.09375
Loss: 0.12777607142925262, 	Error: 0.03125
Loss: 0.1766306310892105, 	Error: 0.0625
Loss: 0.16358394920825958, 	Error: 0.125
Loss: 0.19927041232585907, 	Error: 0.09375
Loss: 0.1349705159664154, 	Err

In [30]:
Error = sess.run(error, feed_dict={gameboards:test_gameboards, labels:test_labels, keep_prob:1.0})

In [31]:
print(Error)

0.468715
